In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Step 1: Load and preprocess the dataset
data = load_iris()
X = data.data
y = data.target
X, y

In [3]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Directory to save models
import os
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)

In [7]:
# Step 2: Train models with different hyperparameters
hyperparams = [
    {"C": 0.1, "max_iter": 100},
    {"C": 1.0, "max_iter": 200},
    {"C": 10.0, "max_iter": 300},
]

results = []

In [ ]:
import pickle
for idx, params in enumerate(hyperparams):
    # Train the model
    model = LogisticRegression(C=params["C"], max_iter=params["max_iter"], random_state=42)
    model.fit(X_train, y_train)

    # Save the model
    model_path = os.path.join(model_dir, f"model_v{idx+1}.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Log the results
    results.append({"version": idx+1, "params": params, "accuracy": accuracy})

    # Step 3: Version the model using Git tags
    os.system(f"git add {model_path}")
    os.system(f"git commit -m 'Add model version {idx+1} with params {params}'")
    os.system(f"git tag -a v{idx+1} -m 'Version {idx+1} with accuracy {accuracy}'")
    os.system(f"git push origin main --tags")

In [ ]:
# Display results
results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
# Save results to a CSV file
results_df.to_csv("model_results.csv", index=False)